In [161]:
import requests
from scrapy.selector import Selector
import re
import os
from dotenv import load_dotenv
import telegram
import time 
load_dotenv()
token = os.getenv('token')
chat_id = os.getenv('chat_id')
user_agent = os.getenv('user_agent')

In [162]:
# список аниме

In [264]:
response = requests.get("https://jut.su/anime/page-1/",
                       headers = {
        'User-Agent': user_agent
    })

In [276]:
content = response.content.decode('cp1251').encode('utf8')
print(type(content))

<class 'bytes'>


In [277]:
x = Selector(text=content).xpath('//div[contains(@class, "all_anime_global")]')

In [278]:
refs = x.xpath('./a/@href').getall()
titles = x.xpath('.//div[contains(@class, "aaname")]/text()').getall()
x_series = x.xpath('.//div[contains(@class, "aailines")]')

def get_series_cnt(content:Selector)->int:
    text = "".join(content.xpath("./text()").getall())
    series_cnt = re.search(r"(\d+)\sсери", text).group(1)
    return int(series_cnt) if series_cnt else 0

series_cnts = [get_series_cnt(item) for item in x_series]

In [279]:
def get_series_cnt_updated(content:Selector)->dict:
    text = "".join(content.xpath("./text()").getall())
    series_match= re.search(r"(\d+)\sсери", text)
    films_match = re.search(r"(\d+)\sфильм", text)
    series_cnt = int(series_match.group(1)) if series_match else 0
    films_cnt = int(films_match.group(1)) if films_match else 0
    return {'series_cnt': series_cnt,'films_cnt': films_cnt}
    
series_cnts = [get_series_cnt_updated(item) for item in x_series]

In [280]:
anime_dict = {}
for ref, title, series_cnt in zip(refs, titles, series_cnts):
    if ref not in anime_dict:
        anime_dict[ref] = {"ref":ref, "title":title, "series_cnt":series_cnt['series_cnt'],'films_cnt':series_cnt['films_cnt']}
anime_dict

{'/shingekii-no-kyojin/': {'ref': '/shingekii-no-kyojin/',
  'title': 'Атака титанов',
  'series_cnt': 87,
  'films_cnt': 2},
 '/full-metal-alchemist/': {'ref': '/full-metal-alchemist/',
  'title': 'Стальной алхимик',
  'series_cnt': 115,
  'films_cnt': 2},
 '/onepunchman/': {'ref': '/onepunchman/',
  'title': 'Ванпанчмен',
  'series_cnt': 24,
  'films_cnt': 0},
 '/sword-arts-online/': {'ref': '/sword-arts-online/',
  'title': 'Мастера меча онлайн',
  'series_cnt': 96,
  'films_cnt': 2},
 '/boku-hero-academia/': {'ref': '/boku-hero-academia/',
  'title': 'Моя геройская академия',
  'series_cnt': 118,
  'films_cnt': 3},
 '/kimetsu-yaiba/': {'ref': '/kimetsu-yaiba/',
  'title': 'Клинок, рассекающий демонов',
  'series_cnt': 44,
  'films_cnt': 0},
 '/tokushu/': {'ref': '/tokushu/',
  'title': 'Токийский Гуль',
  'series_cnt': 48,
  'films_cnt': 0},
 '/hunter-hunter/': {'ref': '/hunter-hunter/',
  'title': 'Хантер х Хантер',
  'series_cnt': 148,
  'films_cnt': 2},
 '/stein-gate/': {'ref': 

In [251]:
#Selection
#selected = ['/full-metal-alchemist/']
#selected_dict = {key:item for key, item in anime_dict.items() if key in selected}

In [281]:
def get_data(url:str)->dict:
    """The function gets url to scrape site content specifically title name,title page source, count of series,count of films'
    from all content pages of site and return dict object
    INPUT:
    URL:url adress must be like https://jut.su/anime/page-(\d)/
    Note: It's also use global constants from .env file  
    
    OUTPUT:
    dict: the dict object which kind is {title_internal_link:
    {'title_internal_link':title internal link,
     'title': title name in Russian,
     'series_cnt': series count,
     'film_cnt': film count
      }}"""
    anime_dict = {}
    page_num = 1
    while True:
        try:
            page_url = url + f"page-{page_num}/"
            print(page_url)
            response = requests.get(page_url,headers={
            "User-Agent": user_agent # global var
                    })
            content = response.content.decode("cp1251").encode('utf-8')
            selections = Selector(text=content).xpath('//div[contains(@class, "all_anime_global")]')
            links = selections.xpath('./a/@href').getall()
            titles_name = selections.xpath('.//div[contains(@class, "aaname")]/text()').getall()
            titles_info = x.xpath('.//div[contains(@class, "aailines")]')
            series_cnts = [get_series_cnt_updated(item) for item in titles_info]
            if len(dic(series_cnts))==0:
                print("shhh")
                raise Exception
            for link, title, series_cnt in zip(links,titles_name,series_cnts):
                 if ref not in anime_dict:
                        anime_dict[link] = {"ref":link, "title":title, "series_cnt":series_cnt['series_cnt'],'films_cnt':series_cnt['films_cnt']}   
            page_num+=1
            del series_cnts
        except Exception as e: 
            print(e)
            return (anime_dict or "No conection to the server")

In [282]:
print(get_data("https://jut.su/anime/"))

https://jut.su/anime/page-1/
https://jut.su/anime/page-2/
https://jut.su/anime/page-3/
https://jut.su/anime/page-4/
https://jut.su/anime/page-5/
https://jut.su/anime/page-6/
https://jut.su/anime/page-7/
https://jut.su/anime/page-8/
https://jut.su/anime/page-9/


KeyboardInterrupt: 

In [114]:
# for key in selected:
#     response = requests.get(f"https://jut.su{key}",
#                        headers = {
#         'User-Agent': user_agent
#     })
#     content = response.content.decode('cp1251').encode('utf8')
#     series = Selector(text=response.content).xpath('//a[contains(@class, "short-btn")]//@href')
#     diff = len(series) - anime_dict[key]["n_series"]
#     if  diff!= 0:
#         name = anime_dict[key]["name"]
#         print(f"{name} {diff} новых серий!")
#     anime_dict[key]["n_series"] = len(series)
# # TODO descrpition
# # x.xpath('.//div[contains(@class, "tooltip_of_the_anime")]/@content').get()

In [10]:
# class AnimeObserver():
#     def __init__(self, selected):
#         self.selected = selected
        

In [11]:
selected_item = selected_dict['/full-metal-alchemist/']

In [ ]:
# from telegram.ext.updater import Updater
# from telegram.update import Update
# from telegram.ext.callbackcontext import CallbackContext
# from telegram.ext.commandhandler import CommandHandler
# from telegram.ext.messagehandler import MessageHandler
# from telegram.ext.filters import Filters
#
# updater = Updater("token", use_context=True)
#
#
# def start(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Hello sir, Welcome to the Bot.Please write\
#         /help to see the commands available.")
#
# def help(update: Update, context: CallbackContext):
#     update.message.reply_text("""Available Commands :-
#     /add - Add selected series
#     /remove - Remove series from selected list
#     """)
#
#
# def add(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         f"{selected_item['name']}: {selected_item['series_cnt']}")
#
# def remove(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Remove items")
#
#
#
#
#
# def unknown(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Sorry '%s' is not a valid command" % update.message.text)
#
#
# def unknown_text(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Sorry I can't recognize you , you said '%s'" % update.message.text)
#
#
# updater.dispatcher.add_handler(CommandHandler('start', start))
# updater.dispatcher.add_handler(CommandHandler('add', add))
# updater.dispatcher.add_handler(CommandHandler('remove', remove))
# updater.dispatcher.add_handler(CommandHandler('help', help))
# updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown))
# updater.dispatcher.add_handler(MessageHandler(
#     Filters.command, unknown)) # Filters out unknown commands
#
# # Filters out unknown messages.
# updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown_text))
#
# updater.start_polling()


In [24]:
try: 
    print("Some str")
    raise Exception
except:
    print("Another str")

Some str
Another str
